# Interactive Web Maps of COVID19 data in Python

## Libraries

For this session, we will be using two libraries: folium and altair. Make sure to install the libraries using `pip` or `conda`. If you have reached this Jupyter notebook via the IDRE GitHub page, or, if you installed Anaconda and ran `requirements.txt`, your library should already be installed. If not, uncomment the install code below and run them.


In [ ]:
# !conda install -c conda-forge folium

Some resources:
* official folium documentation: https://python-visualization.github.io/folium/index.html 
* https://coderzcolumn.com/tutorials/data-science/interactive-maps-choropleth-scattermap-using-folium
* a simple jupyter notebook with examples: https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Features.ipynb
* choropleth mapping: https://coderzcolumn.com/tutorials/data-science/interactive-maps-choropleth-scattermap-using-folium
* working with geopandas: http://andrewgaidus.com/leaflet_webmaps_python/

Import the folium library.

In [ ]:
import folium

Create your first interactive map using the `folium.Map()` command. Notice the following common arguments:
* `location` is an array of [latitute, longitude]
* `zoom_start` is the default zoom level, 0 being the entire planet, 16 being very zoomed in (block level)
* `control_scale` adds a scale to the map

In [ ]:
# Create a Map instance
map = folium.Map(location=[34,-118], 
               zoom_start=8, 
               control_scale=True
                )

map

You can choose to display a different basemap using the `tiles` property:
* `OpenStreetMap`
* `Stamen` (Terrain, Toner, and Watercolor)
* `CartoDB` (positron and dark_matter)

In [ ]:
# Create a Map instance with different basemap
map = folium.Map(location=[34,-118], 
               zoom_start=8, 
               control_scale=True,
               tiles='CartoDB dark_matter',
               attr= '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>')
map

## Vector layers
You can add vector layers (markers, lines, polygons, circles) to the map. Here, we will add a `folium.CircleMarker()`.

https://python-visualization.github.io/folium/modules.html#module-folium.vector_layers

In [ ]:
# add a circle
my_circle = folium.Circle(
    radius=10000, # this is in meters
    location=[34,-118],
    color='crimson',
    fill=True,
)
my_circle.add_to(map)
map

Reset the map by calling and recreating `folium.Map()` once again:

In [ ]:
# reset the map (only way to get rid of circles)
map = folium.Map(location=[34,-118], 
                zoom_start=8, 
                control_scale=True,
                tiles='CartoDB dark_matter',
                attr= '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>')

## Adding data to the interactive map
Now that we know how to create an interactive map, let's add some data using the `pandas` library. We will grab the `latimes-place-totals` data table from the LA Times.

In [ ]:
# get latimes covid19 data
import pandas as pd

latimes = pd.read_csv(
    "https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/latimes-place-totals.csv"
)
latimes


That's a lot of rows. Let's filter it to something more manageable.

In [ ]:
# sort it by date
latimes = latimes.sort_values(by=["date"], ascending=True)

# create a variable for the last date
lastdate = latimes.iloc[-1]['date']
lastdate

In [ ]:
# create a subset dataset for a single day in Los Angeles
latimes_today = latimes.query("date == @lastdate & county == 'Los Angeles'")
latimes_today

Check the data with a simple panda plot.

In [ ]:
latimes_today.plot(x ='x', y='y', kind = 'scatter')

## Loop and map

We have our interactive map. We have our filtered dataset. Now let's map all those rows. To do so, set up a `For` loop:

In [ ]:
# loop through the rows in Los Angeles, and create a circle based on confirmed cases
for index, row in latimes_today.iterrows():
    # set up the variables
    lat = row['y']
    lon = row['x']
    label = str(row['confirmed_cases']) + ' confirmed cases in ' + row['place']
    size = row['confirmed_cases']
    
    # create a circle for every row
    circle = folium.Circle(
        radius=size,
        location=[lat,lon],
        tooltip = label,
        color='crimson',
        fill = True
    )
    circle.add_to(map)
    

In [ ]:
# show the map
map

## Advanced: Let's chart with altair
* altair documentation: https://altair-viz.github.io/getting_started/overview.html

In [ ]:
import altair as alt

In [ ]:
# reset the map (only way to get rid of circles)
map = folium.Map(location=[34,-118], 
                zoom_start=8, 
                control_scale=True,
                tiles='CartoDB dark_matter',
                attr= '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>')

Create a function that creates circles

In [ ]:
# create a function to create circles, and also add a chart in the popup window (a lot here, will break it up later)
def createCircle(lat,lon,size,place,label):
    # create a bar chart for each circle
    bar = alt.Chart(latimes.query('place == @place')).mark_bar().encode(
        x=alt.X('date', axis=alt.Axis(labels=False)), # turn the labels off because there are too many
        y='confirmed_cases',
        color='confirmed_cases',
        tooltip = ['date','place','confirmed_cases']
    ).properties(width=400,height=200)

    # add the bar chart as a folium feature
    vega = folium.features.VegaLite(
        bar,
        width=600,
        height=200,
    )

    # create the circle
    circle = folium.Circle(
        radius=size,
        location=[lat,lon],
        tooltip = label,
        color='crimson',
        fill = True
    )

    # create a popup
    popup = folium.Popup()

    # add the chart to the popup
    vega.add_to(popup)
    
    # add the popup to the circle
    popup.add_to(circle)
    
    # add the circle to the map
    circle.add_to(map)
    

In [ ]:
# loop through the rows in Los Angeles, and create a circle based on confirmed cases
for index, row in latimes_today.iterrows():
    label = str(row['confirmed_cases']) + ' confirmed cases in ' + row['place']
    createCircle(row['y'],row['x'],row['confirmed_cases'],row['place'],label)

In [ ]:
# show the map
map

In [ ]:
# save it!
map.save('index.html')

# Post workshop survey
Please take the following survey if you participated in any part of this workshop.

https://bit.ly/2VkkyXm